In [14]:
import datetime as dt
from bs4 import BeautifulSoup
import requests
import re
import time
import random
import pandas as pd
import os
import numpy as np

In [16]:
master_df = pd.read_csv('week_5.csv')
hist = pd.read_csv('pos_sal_avg.csv')
master_df = master_df.merge(hist, left_on=['Pos','Salary'], right_on=['Pos','DK salary'])

In [17]:
master_df.head(15)

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,DK salary,DK points
0,WR,DeAndre Hopkins,7900,ARI,22.68,20.2,7900.0,18.078261
1,QB,Lamar Jackson,7900,BAL,21.56,24.1,7900.0,33.780000
2,RB,Ezekiel Elliott,7800,DAL,22.55,21.6,7800.0,20.421053
3,QB,Patrick Mahomes,7700,KC,28.54,25.3,7700.0,30.280000
4,WR,Calvin Ridley,7500,ATL,22.38,18.7,7500.0,18.757576
5,QB,Josh Allen,7500,BUF,33.08,22.1,7500.0,22.691111
6,QB,Dak Prescott,7400,DAL,34.30,23.5,7400.0,22.791429
7,WR,Amari Cooper,7400,DAL,22.78,17.8,7400.0,13.325000
8,QB,Kyler Murray,7200,ARI,27.32,22.0,7200.0,18.897500
9,RB,Derrick Henry,7000,TEN,19.17,14.4,7000.0,14.676923


In [18]:
master_df['val_above_sal'] = master_df['Projected Points'] - master_df['DK points']

In [19]:
master_df['Points Per Dollar'] = master_df['Projected Points'] / \
    master_df['Salary']
master_df['Points Per 1K'] = master_df['Projected Points'] / 1000
master_df['Value'] = master_df['Projected Points'] / \
    (master_df['Salary']/1000)

In [20]:
master_df

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,DK salary,DK points,val_above_sal,Points Per Dollar,Points Per 1K,Value
0,WR,DeAndre Hopkins,7900,ARI,22.68,20.2,7900.0,18.078261,2.121739,0.002557,0.0202,2.556962
1,QB,Lamar Jackson,7900,BAL,21.56,24.1,7900.0,33.780000,-9.680000,0.003051,0.0241,3.050633
2,RB,Ezekiel Elliott,7800,DAL,22.55,21.6,7800.0,20.421053,1.178947,0.002769,0.0216,2.769231
3,QB,Patrick Mahomes,7700,KC,28.54,25.3,7700.0,30.280000,-4.980000,0.003286,0.0253,3.285714
4,WR,Calvin Ridley,7500,ATL,22.38,18.7,7500.0,18.757576,-0.057576,0.002493,0.0187,2.493333
...,...,...,...,...,...,...,...,...,...,...,...,...
390,DST,Dolphins,2400,MIA,5.25,6.0,2400.0,6.831683,-0.831683,0.002500,0.0060,2.500000
391,DST,Bengals,2300,CIN,3.25,5.3,2300.0,5.700935,-0.400935,0.002304,0.0053,2.304348
392,DST,Giants,2200,NYG,4.00,6.0,2200.0,5.894737,0.105263,0.002727,0.0060,2.727273
393,DST,Titans,2100,TEN,5.00,6.3,2100.0,4.621951,1.678049,0.003000,0.0063,3.000000


In [21]:
good_choices = master_df[(master_df['val_above_sal']>0) & (master_df['Projected Points']>11)]

In [22]:
good_choices.sort_values(by=['Pos','val_above_sal'], ascending=False)

,Pos,Name,Salary,TeamAbbrev,AvgPointsPerGame,Projected Points,DK salary,DK points,val_above_sal,Points Per Dollar,Points Per 1K,Value
7,WR,Amari Cooper,7400,DAL,22.78,17.8,7400.0,13.325000,4.475000,0.002405,0.0178,2.405405
44,WR,Robby Anderson,5900,CAR,19.68,15.4,5900.0,12.243077,3.156923,0.002610,0.0154,2.610169
33,WR,Tyler Boyd,6200,CIN,17.35,14.3,6200.0,11.572075,2.727925,0.002306,0.0143,2.306452
100,WR,Darius Slayton,4800,NYG,13.15,12.7,4800.0,9.988889,2.711111,0.002646,0.0127,2.645833
37,WR,Terry McLaurin,6100,WAS,19.25,15.5,6100.0,13.235088,2.264912,0.002541,0.0155,2.540984
45,WR,DeVante Parker,5900,MIA,15.22,14.5,5900.0,12.243077,2.256923,0.002458,0.0145,2.457627
0,WR,DeAndre Hopkins,7900,ARI,22.68,20.2,7900.0,18.078261,2.121739,0.002557,0.0202,2.556962
122,WR,Sammy Watkins,4500,KC,11.02,12.0,4500.0,10.109134,1.890866,0.002667,0.0120,2.666667
92,WR,TY Hilton,4900,IND,7.30,12.6,4900.0,10.839583,1.760417,0.002571,0.0126,2.571429
116,WR,Golden Tate,4600,NYG,8.10,11.5,4600.0,9.935849,1.564151,0.002500,0.0115,2.500000


In [33]:
Tuesday = master_df[master_df['Name'].isin(['Titans ','Patrick Mahomes',
                'David Johnson', 'Ezekiel Elliott', 'Tyreek Hill', 
                'Sammy Watkins', "Damiere Byrd", 'Travis Kelce', "Mike Davis"])]
Wednesday = master_df[master_df['Name'].isin(['Titans ','Patrick Mahomes',
                'David Johnson', 'Ezekiel Elliott', 'Robby Anderson', 
                'DJ Moore', "Damiere Byrd", 'Travis Kelce', "Darren Waller"])]
Thursday = master_df[master_df['Name'].isin(['Titans ','Patrick Mahomes',
                'David Johnson', 'Ezekiel Elliott', 'Tyreek Hill', 
                'Sammy Watkins', "Damiere Byrd", 'Travis Kelce', "Mike Davis"])]
Friday = master_df[master_df['Name'].isin(['Titans ','Patrick Mahomes',
                'David Johnson', 'Ezekiel Elliott', 'Tyreek Hill', 
                'Sammy Watkins', "Damiere Byrd", 'Travis Kelce', "Mike Davis"])]

In [34]:
print(Tuesday['Projected Points'].sum(),Tuesday['Salary'].sum())

141.4 50000


In [35]:
print(Wednesday['Projected Points'].sum(),Wednesday['Salary'].sum())

141.10000000000002 50000


In [36]:
print(Thursday['Projected Points'].sum(),Thursday['Salary'].sum())

141.4 50000


In [37]:
print(Friday['Projected Points'].sum(),Friday['Salary'].sum())

141.4 50000
